## Lesson 05 - The Final Report

#### Overview: 

In this lesson we're going to put together a simple summary report with a roll-up of the data we've looked at so far and a basic graph to show you how to interact with Excel's graphs.

#### Handy References:
* [Official Python Documentation](https://docs.python.org/3/)
* [Jupyter Notebook Documentation](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html)
* [Pandas](https://pandas.pydata.org/)
* [XlsxWriter](https://xlsxwriter.readthedocs.io/)

### The Data

Rather than to through the process of reading and formatting all the previous files we've looked at, I've put them into a single file called 'combined_report.xlsx' with four sheets: Sales, Billing, Supply, and HR.  We're going to read all four sheets into our notebook, do some formatting and clean-up, then combine them into a single DataFrame.

In [ ]:
# File Imports
import pandas as pd
import xlsxwriter
import os

In [ ]:
# Define the path to the file
combined_report = os.path.join('..', 'data', 'combined_report.xlsx')

In [ ]:
# Read the file into our notebook
frames = pd.read_excel(combined_report, sheet_name=None)

In [ ]:
# Summarize the Sales Data
sales_summary = pd.DataFrame(frames['Sales']).groupby(['Month'], as_index=False).agg({
    'Amt': [sum],
    'Commission': [sum]
})

In [ ]:
# Rename the columns
sales_summary.columns = ['Month', 'Sales Amount', 'Sales Commission']

#### Categorical Data
* To make it easier to sort our months without changing them to date-times and back again, we're going to use [pd.Categorical](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to assign the 'Month' column as a Categorical data type and manually define how it should be sorted.
* Then we'll use [sort_values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) to sort our table in the order specified.

In [ ]:
# Define the category
sales_summary['Month'] = pd.Categorical(sales_summary['Month'], ['March', 'April', 'May'])

In [ ]:
# Sort the values
sales_summary.sort_values(by=['Month'], inplace=True)

In [ ]:
# Preview our data
sales_summary.head()

Now we'll use that same method to build our billing, supply, and HR tables: 

In [ ]:
billing_summary = pd.DataFrame(frames['Billing']).groupby(['Month'], as_index=False).agg({
    'Total Billed': [sum],
    'Total Paid': [sum],
})

In [ ]:
billing_summary.columns = ['Month', 'Amount Billed', 'Amount Paid']

In [ ]:
billing_summary['Month'] = pd.Categorical(billing_summary['Month'], ['March', 'April', 'May'])

In [ ]:
billing_summary.sort_values(by=['Month'], inplace=True)

In [ ]:
billing_summary['Outstanding Balance'] = billing_summary['Amount Billed'] - billing_summary['Amount Paid']

In [ ]:
billing_summary.head()

In [ ]:
supply_summary = pd.DataFrame(frames['Supply']).groupby(['Month'], as_index=False).agg({
    'Incoming': [sum],
    'Outgoing': [sum],
    'On-Hand': [sum],
})

In [ ]:
supply_summary.columns = ['Month', 'Supply - Incoming', 'Supply - Outgoing', 'Supply - On-Hand']

In [ ]:
supply_summary['Month'] = pd.Categorical(supply_summary['Month'], ['March', 'April', 'May'])

In [ ]:
supply_summary.sort_values(by=['Month'], inplace=True)

In [ ]:
supply_summary.head()

In [ ]:
hr_summary = pd.DataFrame(frames['HR']).groupby(['Month'], as_index=False).agg({
    'Complaints Received': [sum],
})

In [ ]:
hr_summary.columns = ['Month', 'HR Incidents']

In [ ]:
hr_summary['Month'] = pd.Categorical(hr_summary['Month'], ['March', 'April', 'May'])

In [ ]:
hr_summary.sort_values(by=['Month'], inplace=True)

In [ ]:
hr_summary.head()

#### Horizontal Concatenation
* We've used `pd.concat()` to combine multiple DataFrames in previous lessons.  
* We're going to use that same method here, only with the added argument of `axis=1` to specify that we want to combine them horizontally. 
* We will then `transpose()` the combined DataFrame so we can drop the duplicated 'Month' column.
* Lastly, we'll `transpose()` the DataFrame a final time to bring it back to its original state.

In [ ]:
summary_df = pd.concat([sales_summary, billing_summary, supply_summary, hr_summary], axis=1).\
                T.drop_duplicates().T

In [ ]:
summary_df.head()

In [ ]:
summary_df.info()

#### Numeric Types
* If you look at the output from `.info()` you'll see that our number columns are objects and not numbers.
* We're going to use [pd.to_numeric()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html#pandas.to_numeric) to convert their type so that we can get a "Total" row.
* We'll do that by passing a list of our columns to `pd.to_numeric()` wwith the `errors='ignore'` argument.  This ensures that non-numeric columns like 'Month' are not effected.

In [ ]:
# Get a list of columns in our DataFrame
cols = summary_df.columns

In [ ]:
# Convert the columns to a Numeric type
summary_df[cols] = summary_df[cols].apply(pd.to_numeric, errors='ignore')

In [ ]:
# Verify the type conversion worked
summary_df.info()

#### Getting a Total Row
* We want the bottom row of our DataFrame to be a 'Total Row'.
* We will do this by setting the index to the 'Month' Column so we can add a new index entry
* Then we'll create a new index entry with `df.loc[row_name]` and assign the row's values to the [sum()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sum.html) of the previous rows.
* Finally, we'll use `reset_index()` to put the 'Month' column back where it should be.

In [ ]:
# Set 'Month' as the index
summary_df.set_index('Month', inplace=True)

In [ ]:
summary_df.head()

In [ ]:
# Create a 'Total' row
summary_df.loc["Total"] = summary_df.sum()

In [ ]:
# Reset the index
summary_df.reset_index(inplace=True)

In [ ]:
# Now let's preview our data
summary_df.head()

### Writing to Excel
Most of this should look familiar.  We're going to write our DataFrame to Excel like we've done in the past, with a few minor changes.  What's really new is we're going to add in some custom header text above the DataFrame and add in a simple bar chart.

#### Define the Output File:

In [ ]:
output_file = os.path.join('..', 'data', 'Quarterly Report.xlsx')
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
workbook = writer.book

#### Define the formats:
* We have a couple of new formats: `title_format` and `subheader_format`.  By now you should be familiar with how these are put together. 

In [ ]:
# Define the format for our header:
title_format = workbook.add_format({
    'bold': True, 
    'align': 'center', 
    'valign': 'top',
    'fg_color': '#4472C1',
    'font_color': 'white',
    'font_size': 16,
})

header_format = workbook.add_format({
    'bold': True,
    'align': 'center',
    'valign': 'top',
    'fg_color': '#4472C1',
    'font_color': 'white',
    'font_size': 12,
})

# Define the subheader format:
subheader_format = workbook.add_format({
            'bold': False,
            'align': 'center',
            'valign': 'top',
            'fg_color': '#8EA9DB',
            'font_color': 'black',
            'text_wrap': 1,
            'border': 1})

# Define the format for our numbers:
number_format = workbook.add_format({'num_format': '#,##0'})
accounting_format = workbook.add_format({'num_format': '$#,##0'})

#### Define our Worksheet and Write to Excel
* Notice we've added `startrow=3` and `header=False` as arguments here.
* We're doing this to leave room for the extra text we will add to the top of the worksheet later.

In [ ]:
sheet = 'Summary'
summary_df.to_excel(writer, sheet_name=sheet, startrow=3, index=False, header=False)

In [ ]:
worksheet = writer.sheets[sheet]

In [ ]:
for col_num, value in enumerate(summary_df.columns.values):
    worksheet.write(2, col_num, value, header_format)

#### Column Formatting:
* In the past we've had to manually figure out which Excel column belonged to which DataFrame Column.
* The method I use below gets us the column number for our columns using Python's build-in .index() method on our column list.

In [ ]:
# Get the column number for month
month_column = list(summary_df.columns).index('Month')

# Get the column numbers for accounting columns
accounting_start = list(summary_df.columns).index('Sales Amount')
accounting_end = list(summary_df.columns).index('Outstanding Balance')

# Get the number column numbers
num_start = list(summary_df.columns).index('Supply - Incoming')
num_end = list(summary_df.columns).index('HR Incidents')

Using those column numbers, we can set our column formatting based on the numbers of the first and last columns we want the formatting applied to: 

In [ ]:
# Set the column formats

# Month Column
worksheet.set_column(month_column, month_column, 14, None)

# Accounting Columns
worksheet.set_column(accounting_start, accounting_end, 14, accounting_format)

# Number Columns
worksheet.set_column(num_start, num_end, 14, number_format)

#### Writing Raw Text to Excel:
* Most of the time, we're not just writing tables to Excel.  We need text, headers, labels, etc.
* You can write text to Excel using [worksheet.write()](https://xlsxwriter.readthedocs.io/worksheet.html?highlight=worksheet.write).
* However, for this case we need to write to a merged range of cells.  We do that with [worksheet.merge_range()](https://xlsxwriter.readthedocs.io/worksheet.html?highlight=worksheet.merge_range()).  
* `merge_range` works by defining the start row, start column, stop row, and stop column, then the text we want to write, then the format we want applied:

In [ ]:
# Writing the Headers
worksheet.merge_range(0, 0, 0, len(summary_df.columns)-1, 'Dunder Mifflin Scranton', title_format)
worksheet.merge_range(1, 0, 1, len(summary_df.columns)-1, 'Quarterly Summary', subheader_format)

#### Excel Charts
* Xlsxwriter can create any chart that Excel can create.  The charting functionality is extensive and could span multiple classes all on its own.
* For this class we'll get a brief introduction to charting by making a simple bar graph.  
* I encourage you to take a look at [The Chart Class](https://xlsxwriter.readthedocs.io/chart.html) documentation to get a feel for just how powerful this can be.

#### Defining the Chart:

In [ ]:
simple_chart = workbook.add_chart({'type': 'column'})

#### Setting the Data Range:
* We'll start by defining the columns we want to include in the chart
* Then we'll define the column we want to use for charting.  In this case it's the first column, or zero.
* Next we will define the rows we want included.  In our case, the table starts on row 3 and I want to stop just before the total row, or row 5.

In [ ]:
chart_columns = ['Sales Amount', 'Amount Billed', 'Amount Paid']
category_column = 0
first_row = 3
last_row = 5

#### Adding Data to the Chart
* We will use a `For` loop to iterate through our `chart_columns`
* For each column:
* * Get the column number
* * Add the data from that column using `add_series()`

* `add_series()` needs:
* * Series Name: `name`
* * Categories: `categories` - The category labels for our series.  This is a list containing the source sheet, first row, first column, last row, and last column of our categories.  In our case, we are using the category column.
* * Values: `values` - The data we want to add to the chart.  This is similiar to `categories`.

In [ ]:
# Remember, for categories and values the order is [sheetname, first_row, first_col, last_row, last_col]
for col in chart_columns:
    # Get the column number of the data we want to add
    col_num = list(summary_df.columns).index(col)
    # Add the series
    simple_chart.add_series({
        'name': col,
        'categories': [sheet, first_row, category_column, last_row, category_column],
        'values': [sheet, first_row, col_num, last_row, col_num]
    })

In [ ]:
# Set the chart title
simple_chart.set_title({'name': 'Simple Bar Chart'})

#### Inserting the Chart
* You can add the chart to the workbook using `insert_chart` and passing the cell where you want the chart to begin.

In [ ]:
worksheet.insert_chart('A9', simple_chart)

In [ ]:
# Save the file
writer.save()

### Exercise - Making a Bar Chart
* For this exercise, you'll be reading in the "Expense Summary" sheet from "accounting_summary_master.xlsx".
* Once the file is read into our notebook, do the following:
* * Summarize the Expenses by Employee and get their total Amount Paid
* * Sort the new DataFrame by Amount Paid in Descending Order
* * Rename the columns to "Employee" and "Amount Paid"
* * Write the new DataFrame to Excel with "Amount Paid" having an accounting format
* * Insert a bar chart in cell C1
* * Save the file

In [ ]:
# Define the path to the file
accounting_file = os.path.join(_____, 'data', _____)

In [ ]:
# Read the 'Expense Summary' sheet into a DataFrame
expense_df = pd._____(accounting_file, sheet_name='Expense Summary')

In [ ]:
# Create a summary DataFrame grouped by 'Employee' with the total 'Amount Paid' for each
summary_df = expense_df._____([_____], as_index=False).agg({
    'Amount Paid': [_____]
})

In [ ]:
# Rename the columns
summary_df._____ = ['Employee', 'Amount Paid']

In [ ]:
# Sort by 'Amount Paid' in descending order
summary_df._____(by=['Amount Paid'], ascending=_____, inplace=True)

In [ ]:
# Define the output file and the writer
output_file = __.____.____('..', 'data', 'Expense Report.xlsx')
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
workbook = writer.book

In [ ]:
# Define the format for our header:
header_format = workbook.add_format({
    'bold': True,
    'align': 'center',
    'valign': 'top',
    'fg_color': '#4472C1',
    'font_color': 'white',
    'font_size': 12,
})

# Define the format for Amount Paid
accounting_format = workbook.add_format({_____: '$#,##0'})

In [ ]:
# Define the sheet
sheet = 'Summary'
summary_df._____(writer, sheet_name=sheet, index=False)

In [ ]:
# Define the worksheet
worksheet = writer._____[sheet]

In [ ]:
# Write the data to Excel
for col_num, value in enumerate(_____.columns.values):
    worksheet.write(0, col_num, value, _____)

In [ ]:
# Apply the accounting format to the 'Amount Paid' column
worksheet.________('B:B', 14, accounting_format)

In [ ]:
# Define the chart
simple_chart = workbook.________({'type': 'column'})

In [ ]:
# Define some chart variables
category_column = 0
first_row = 1
last_row = 9

In [ ]:
# Add data to the chart
# Remember, for categories and values the order is [sheetname, first_row, first_col, last_row, last_col]

# Define the 'Amount Paid' column
amount_column = _____

# Get the column number for the 'Amount Paid' column
col_num = ____(summary_df.columns)._____(amount_column)

# Add the series to the chart
simple_chart.add_series({
    'name': ______,
    'categories': [sheet, _____, category_column, _____, category_column],
    'values': [sheet, _____, col_num, _____, col_num]
    })

In [ ]:
# Set the chart title
simple_chart._____({'name': 'Expense Chart'})

In [ ]:
# Insert the chart into the sheet
worksheet._____(_____, simple_chart)

In [ ]:
# Save the file
writer.save()